In [1]:
city = 'beijing'
learning_rate = 0.001
car_id_batch_size = 5
keep_probability = 1
gap = 20
predict_steps = 10
n_hidden_1 = 56
n_hidden_2 = 48
n_dense = [128,128] # = []
epochs = 500
model_index = 1

# encoder1st_model_index,
# encoder2nd_model_index = 90
load_model_index = None
display_step = 10
r_threshold = 200
GPU_index = 0
mu = 1

n_input= 30*2
car_id_batch = int(10000 / car_id_batch_size)
model_name = 'SAE_all_'+ city + '_' +str(model_index)
n_output= predict_steps*2


import tensorflow as tf
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU_index)
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from math import ceil,sqrt

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True  # 设置tf模式为按需赠长模式

from geopy.distance import great_circle

def distance(true_x,true_y,predict_x,predict_y):
    return great_circle((true_x,true_y),(predict_x,predict_y)).m

# 向量化
vec_distance = np.vectorize(distance)

print (city+'***********************************************************************************************')
try:
    cdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/middata/'+city+'_tongyi_cdf1.csv',index_col=0)
    ctdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/middata/'+city+'_tongyi_ctdf1.csv',index_col=0)
except:
    cdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/'\
                      +city+'_tongyi_cdf1.csv',index_col=0)
    ctdf = pd.read_csv('/home/liheyuan/Jupyter/trajectory_prediction/trajectory_prediction_1/HMM/hmm4/middata/'\
                       +city+'_tongyi_ctdf1.csv',index_col=0)


tmp_cdf = np.array([cdf.x,cdf.y]).T
min_max_scaler = MinMaxScaler().fit(tmp_cdf)
tmp_1 = min_max_scaler.transform(tmp_cdf)
cdf['x'] = tmp_1[:,0]
cdf['y'] = tmp_1[:,1]

tmp_ctdf = np.array([ctdf.x,ctdf.y]).T
tmp_2 = min_max_scaler.transform(tmp_ctdf)
ctdf['x'] = tmp_2[:,0]
ctdf['y'] = tmp_2[:,1]

print ('data has been imported')

weights = { 
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]),name='W_encoder_h1'),  
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W_encoder_h2'),  
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
}  
biases = {  
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1]),name='b_encoder_h1'),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2]),name='b_encoder_h2'),  
    'decoder_b1': tf.Variable(tf.random_normal([n_input])),  
    'decoder_b2': tf.Variable(tf.random_normal([n_hidden_1])),  
} 

#************************* 1st hidden layer **************  

X = tf.placeholder("float32", [None, n_input])  
y_ = tf.placeholder("float32", shape = [None, n_output])  
keep_prob = tf.placeholder("float32")  

# 编码器1
h1_out =tf.nn.sigmoid(tf.add(tf.matmul(X, weights['encoder_h1']),biases['encoder_b1']))
h1_out_drop = tf.nn.dropout(h1_out,keep_prob)

# 解码器1  
X_1 = tf.nn.sigmoid(tf.matmul(h1_out_drop,weights['decoder_h1'])+biases['decoder_b1'])  
cross_entropy_1 = tf.reduce_mean(-tf.reduce_sum(X * tf.log(X_1),reduction_indices=[1]))
original_loss_1 = tf.reduce_mean(tf.pow(X - X_1, 2))
loss1 = original_loss_1 + mu * cross_entropy_1
train_step_1 = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss1,var_list=[weights['encoder_h1'],biases['encoder_b1'],
                                                                               weights['decoder_h1'],biases['decoder_b1']])  

#************************** 2nd hidden layer *************  

# 编码器2
h2_out =tf.nn.sigmoid(tf.add(tf.matmul(h1_out_drop, weights['encoder_h2']),biases['encoder_b2']))
h2_out_drop = tf.nn.dropout(h2_out,keep_prob)

# 解码器2
X_2 = tf.nn.sigmoid(tf.matmul(h2_out_drop,weights['decoder_h2'])+biases['decoder_b2'])  
cross_entropy_2 = tf.reduce_mean(-tf.reduce_sum(h1_out_drop * tf.log(X_2),reduction_indices=[1]))
original_loss_2 = tf.reduce_mean(tf.pow(h1_out_drop - X_2, 2))
loss2 = original_loss_2 + mu * cross_entropy_2
train_step_2 = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss2,var_list=[weights['encoder_h2'],biases['encoder_b2'],
                                                                               weights['decoder_h2'],biases['decoder_b2']])

#************************** dense layer *************  

with tf.variable_scope('dense_layers'):
    layer_3 = tf.layers.dense(inputs=h2_out_drop, units=128, activation=tf.nn.relu) 
    layer_4 = tf.layers.dense(inputs=layer_3, units=128, activation=tf.nn.relu) 
    logits = tf.layers.dense(inputs=layer_4, units=predict_steps*2)

tmp = y_ - logits
tmp_1 = tf.strided_slice(tmp,[0,0],[(799 - 30 - predict_steps)*car_id_batch_size,predict_steps*2],[1,2])
tmp_2 = tf.strided_slice(tmp,[0,1],[(799 - 30 - predict_steps)*car_id_batch_size,predict_steps*2],[1,2])

loss3 = tf.reduce_mean(tf.sqrt(tf.square(tmp_1)+tf.square(tmp_2)))

init = tf.global_variables_initializer()

# *********************************** gradient check *********************************************

opt = tf.train.GradientDescentOptimizer(learning_rate)

# dense
params_dense = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='dense_layers')

gradients_dense = tf.gradients(loss3, params_dense)

checked_grad_dense = []
for check in gradients_dense:
    checked_grad_dense.append(tf.check_numerics(check,"error occur" ))

with tf.control_dependencies(checked_grad_dense):
    train_step_dense = opt.apply_gradients(zip(checked_grad_dense, params_dense))

# fine
params_ = params_dense[:]
params_.extend([weights['encoder_h1'],biases['encoder_b1'],
    weights['encoder_h2'],biases['encoder_b2']])

gradients = tf.gradients(loss3, params_)

checked_grad = []
for check in gradients:
    checked_grad.append(tf.check_numerics(check,"error occur" ))
    
with tf.control_dependencies(checked_grad):
    train_step_3 = opt.apply_gradients(zip(checked_grad, params_))
    
    
try:
    sess = tf.Session(config=config)
except:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(1-GPU_index)
    sess = tf.Session(config=config)

sess.run(init)

# #     SAE_encoder2nd
# load_model = '../../SAE_2/models/SAE_encoder2nd_'+city+'_'+str(encoder2nd_model_index)+'/save_net.ckpt'
# load_saver = tf.train.Saver({"W_encoder_h1":weights['encoder_h1'], "b_encoder_h1": biases['encoder_b1'],
#                             "W_encoder_h2":weights['encoder_h2'], "b_encoder_h2": biases['encoder_b2']})
# load_saver.restore(sess, load_model)
# print 'SAE_encoder2nd load from '+ load_model


# if load_model_index:
#     load_model = '../models/SAE_encoder2nd_'+city+'_'+str(load_model_index)+'/save_net.ckpt'
#     load_saver = tf.train.Saver()
#     load_saver.restore(sess, load_model)
#     print 'load from '+ load_model
# else:
#     print 'start a new model'

print ('************************************ training the lst layer... ************************************')
loss_list_1 = []
for i in tqdm(range(epochs)):
    tmp_loss_list_1 = []
    for car_id in range(car_id_batch):
        fff = np.array([cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['x'],
                        cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['y']]).T.reshape((car_id_batch_size,800,2))
        batch_x = np.array([fff[:,start_point:start_point+30,:] for start_point in range(0,799 - 30 - predict_steps,predict_steps)])\
                  .reshape((int(ceil(((799 - 30 - predict_steps+1)*1.0/predict_steps)))*car_id_batch_size,30*2))
        
        _,c=sess.run([train_step_1,loss1],feed_dict={X:batch_x, keep_prob:keep_probability})
        tmp_loss_list_1.append(c)
    loss_list_1.append(np.mean(tmp_loss_list_1))
    if (i+1) % display_step == 0:
        print 'loss:%10f'%(loss_list_1[-1])
        
        
print ('************************************ training the 2st layer... ************************************')
loss_list_2 = []
for i in tqdm(range(epochs)):
    tmp_loss_list_2 = []
    for car_id in range(car_id_batch):
        fff = np.array([cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['x'],
                        cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['y']]).T.reshape((car_id_batch_size,800,2))
        batch_x = np.array([fff[:,start_point:start_point+30,:] for start_point in range(0,799 - 30 - predict_steps,predict_steps)])\
                  .reshape((int(ceil(((799 - 30 - predict_steps+1)*1.0/predict_steps)))*car_id_batch_size,30*2))

        _,c=sess.run([train_step_2,loss2],feed_dict={X:batch_x, keep_prob:keep_probability})
        tmp_loss_list_2.append(c)
    loss_list_2.append(np.mean(tmp_loss_list_2))
    if (i+1) % display_step == 0:
        print 'loss:%10f'%(loss_list_2[-1])


print ('************************************ training the dense layer... ************************************')
loss_list_3 = []
for i in tqdm(range(epochs)):
    tmp_loss_list_3 = []
    for car_id in range(car_id_batch):
        fff = np.array([cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['x'],
                        cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['y']]).T.reshape((car_id_batch_size,800,2))
        batch_x = np.array([fff[:,start_point:start_point+30,:] for start_point in range(0,799 - 30 - gap,gap)][:-1])\
                  .reshape((int(ceil(((799 - 30 - gap+1)*1.0/gap))-1)*car_id_batch_size,30*2))
        batch_y = np.array([fff[:,start_point:start_point+predict_steps,:] for start_point in range(29+1,799-gap,gap)][:-1])\
                  .reshape((int(ceil(((799 - 30 - gap+1)*1.0/gap))-1)*car_id_batch_size,predict_steps*2))
        try: 
            _,loss = sess.run([train_step_dense,loss3],feed_dict={X:batch_x,y_:batch_y, keep_prob:keep_probability})
        except tf.errors.InvalidArgumentError:
            continue

        tmp_loss_list_3.append(loss)
    loss_list_3.append(np.mean(tmp_loss_list_3))
    if (i+1) % display_step == 0:
        print 'loss:',loss
        
        
print ('************************************ fine turning... ************************************')
loss_list_4 = []
for i in tqdm(range(epochs)):
    tmp_loss_list_4 = []
    for car_id in range(car_id_batch):
        fff = np.array([cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['x'],
                        cdf.iloc[800*car_id:800*(car_id+car_id_batch_size)]['y']]).T.reshape((car_id_batch_size,800,2))
        batch_x = np.array([fff[:,start_point:start_point+30,:] for start_point in range(0,799 - 30 - gap,gap)][:-1])\
                  .reshape((int(ceil(((799 - 30 - gap+1)*1.0/gap))-1)*car_id_batch_size,30*2))
        batch_y = np.array([fff[:,start_point:start_point+predict_steps,:] for start_point in range(29+1,799-gap,gap)][:-1])\
                  .reshape((int(ceil(((799 - 30 - gap+1)*1.0/gap))-1)*car_id_batch_size,predict_steps*2))
        try: 
            _,loss = sess.run([train_step_3,loss3],feed_dict={X:batch_x,y_:batch_y, keep_prob:keep_probability})
        except tf.errors.InvalidArgumentError:
            continue

        tmp_loss_list_4.append(loss)
    loss_list_4.append(np.mean(tmp_loss_list_4))
    if (i+1) % display_step == 0:
        print 'loss:',loss

f = plt.figure(figsize=(16,5))
plt.plot(loss_list_3)
# plt.savefig('../images/loss_'+model_name+'.png')
plt.show()

f = plt.figure(figsize=(16,5))
plt.plot(loss_list_4)
plt.show()

saver = tf.train.Saver()
save_path = saver.save(sess, '../models/'+model_name+'/save_net.ckpt')
print("Save to path: ", save_path)

print ('beijing predicting vs ...')

predict_positions = []
anwser_positions = []
for car_id in tqdm(range(10000)):
    ttt = np.array([ctdf.iloc[200*car_id:200*(car_id+1)]['x'],
                    ctdf.iloc[200*car_id:200*(car_id+1)]['y']]).T.reshape((200,2))
    batch_x = np.array([ttt[start_point:start_point+30,:] for start_point in range(199 - 30 - predict_steps)])\
              .reshape((199 - 30 - predict_steps,30*2))
    batch_y = np.array([ttt[start_point:start_point+predict_steps,:] for start_point in range(29+1,199-predict_steps)])\
              .reshape((199 - 30 - predict_steps,predict_steps*2))
    predict_positions.append(sess.run(logits,feed_dict={X:batch_x,keep_prob:1.0}))
    anwser_positions.append(batch_y)

predict_positions = np.array(predict_positions).reshape(10000*(199 - 30 - predict_steps)*predict_steps,2)
anwser_positions = np.array(anwser_positions).reshape(10000*(199 - 30 - predict_steps)*predict_steps,2)

# 还原经纬度，用于计算误差
predict_positions = min_max_scaler.inverse_transform(predict_positions)
anwser_positions = min_max_scaler.inverse_transform(anwser_positions)

errors = vec_distance(predict_positions[:,0],predict_positions[:,1],anwser_positions[:,0],anwser_positions[:,1])
mean_error = np.mean(errors)
acc = len(errors[errors<=r_threshold]) * 1.0 / len(errors)

print 'loss:',loss_list_3[-1]
print ''
print mean_error
print acc

beijing***********************************************************************************************
data has been imported
************************************ training the lst layer... ************************************


HBox(children=(IntProgress(value=0, max=500), HTML(value=u'')))

loss:  0.347165
loss:  0.295270
loss:  0.278825
loss:  0.270724
loss:  0.265891
loss:  0.262688
loss:  0.260407
loss:  0.258697
loss:  0.257363
loss:  0.256295
loss:  0.255424
loss:  0.254700
loss:  0.254089
loss:  0.253566
loss:  0.253113
loss:  0.252716
loss:  0.252366
loss:  0.252055
loss:  0.251778
loss:  0.251529
loss:  0.251304
loss:  0.251101
loss:  0.250915
loss:  0.250746
loss:  0.250592
loss:  0.250451
loss:  0.250321
loss:  0.250202
loss:  0.250092
loss:  0.249990
loss:  0.249896
loss:  0.249809
loss:  0.249728
loss:  0.249652
loss:  0.249581
loss:  0.249514
loss:  0.249451
loss:  0.249392
loss:  0.249336
loss:  0.249283
loss:  0.249233
loss:  0.249185
loss:  0.249140
loss:  0.249096
loss:  0.249054
loss:  0.249015
loss:  0.248977
loss:  0.248940
loss:  0.248905
loss:  0.248872

************************************ training the 2st layer... ************************************


HBox(children=(IntProgress(value=0, max=500), HTML(value=u'')))

loss:  0.422042
loss:  0.384358
loss:  0.372346
loss:  0.365744
loss:  0.361460
loss:  0.357634
loss:  0.351653
loss:  0.348501
loss:  0.346709
loss:  0.345257
loss:  0.344046
loss:  0.343018
loss:  0.342131
loss:  0.341355
loss:  0.340665
loss:  0.340041
loss:  0.339473
loss:  0.338947
loss:  0.338455
loss:  0.337990
loss:  0.337551
loss:  0.337135
loss:  0.336743
loss:  0.336377
loss:  0.336034
loss:  0.335712
loss:  0.335411
loss:  0.335127
loss:  0.334858
loss:  0.334602
loss:  0.334357
loss:  0.334123
loss:  0.333899
loss:  0.333683
loss:  0.333475
loss:  0.333274
loss:  0.333080
loss:  0.332891
loss:  0.332709
loss:  0.332532
loss:  0.332360
loss:  0.332192
loss:  0.332028
loss:  0.331869
loss:  0.331712
loss:  0.331559
loss:  0.331409
loss:  0.331262
loss:  0.331117
loss:  0.330974

************************************ training the dense layer... ************************************


...

<Figure size 1600x500 with 1 Axes>

<Figure size 1600x500 with 1 Axes>

('Save to path: ', '../models/SAE_all_beijing_1/save_net.ckpt')
beijing predicting vs ...


HBox(children=(IntProgress(value=0, max=10000), HTML(value=u'')))


loss: 0.0107466355

275.5620343574105
0.617198238994


In [2]:
import numpy as np
def distance(true_x,true_y,predict_x,predict_y):
    return sqrt((true_x-predict_x)**2+(true_y-predict_y)**2)

# 向量化
vec_distance_m = np.vectorize(distance)

error_m = np.mean(vec_distance_m(predict_positions[:,0],predict_positions[:,1],anwser_positions[:,0],anwser_positions[:,1]))

print error_m

0.0032732289975660094
